In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time

# Get jobs

In [3]:
source = requests.get('https://th.jobsdb.com/th').text
soup = BeautifulSoup(source,'html.parser')
print(soup.prettify())

<Response [200]>


TypeError: object of type 'Response' has no len()

In [13]:
html_list = soup.find_all('a',class_="clearfix")
x = html_list[0].prettify()
print(re.search("job-list/(.*?)/(.*?)/",x))

<re.Match object; span=(30, 56), match='job-list/accounting/audit/'>


In [14]:
html_list[0].prettify()

'<a class="clearfix" href="/th/job-list/accounting/audit/1?JSSRC=HPJC" title="Audit">\n <span class="namewrap clearfix">\n  <span>\n   Audit\n  </span>\n </span>\n</a>\n'

In [15]:
x[30:56]

'job-list/accounting/audit/'

In [16]:
#For regular expression testing
# p = re.compile("job-list/(.*?)/(.*?)/")
# p.findall(x)

In [17]:
p = re.compile("job-list/(.*?)/(.*?)/")
job_category_df = pd.DataFrame(columns = ['job_category','job_subcategory'])
for i,html in enumerate(html_list):
    print(p.findall(html.prettify()))
    job = p.findall(html.prettify())
    if job == []:
        break
    job_category_df.loc[i] = [job[0][0],job[0][1]]
    

[('accounting', 'audit')]
[('accounting', 'credit-control')]
[('accounting', 'general-accounting')]
[('accounting', 'taxation')]
[('accounting', 'others')]
[('admin-hr', 'administration-operation-clerical-support')]
[('admin-hr', 'compensation-benefits')]
[('admin-hr', 'employee-relations')]
[('admin-hr', 'recruitment-executive-search')]
[('admin-hr', 'secretary-personal-assistant')]
[('admin-hr', 'training-development')]
[('admin-hr', 'others')]
[('banking-finance', 'analysis')]
[('banking-finance', 'asset-management')]
[('banking-finance', 'bancassurance')]
[('banking-finance', 'corporate-banking')]
[('banking-finance', 'corporate-finance')]
[('banking-finance', 'credit-analysis-approval')]
[('banking-finance', 'credit-management')]
[('banking-finance', 'equities-capital-markets')]
[('banking-finance', 'financial-services')]
[('banking-finance', 'fund-management')]
[('banking-finance', 'investment')]
[('banking-finance', 'loan')]
[('banking-finance', 'mortgage')]
[('banking-finance',

In [18]:
job_category_df.head(10)

,job_category,job_subcategory
0,accounting,audit
1,accounting,credit-control
2,accounting,general-accounting
3,accounting,taxation
4,accounting,others
5,admin-hr,administration-operation-clerical-support
6,admin-hr,compensation-benefits
7,admin-hr,employee-relations
8,admin-hr,recruitment-executive-search
9,admin-hr,secretary-personal-assistant


# Get Salary

In [47]:
# # For test html
# test_html = html_list[0].prettify()
# p = re.compile('href="/(.*?)\?')
# p.findall(test_html)[0]

In [48]:
# # get the first page of all job post of all job categories
# url_list =[]
# for html in html_list:
#     url = 'https://th.jobsdb.com/'
#     link = str(p.findall(html.prettify())[0])
# #     if 'job-list' not in link:
# #         break
#     url_list.append(url+link)
# url_list

In [22]:
# Accounting --> Audit page 1 Link for branded
source = requests.get('https://th.jobsdb.com/th/job-list/accounting/audit/1').text
soup = BeautifulSoup(source,'html.parser')
soup.find_all('div',class_="result-sherlock-cell branded")[0].find('a',class_ = 'posLink',href=True)['href']

'https://th.jobsdb.com/th/en/job/accountant-300003002200652'

In [23]:
# Accounting --> Audit page 1 Link for branded plus
soup.find_all('div',class_="result-sherlock-cell branded plus")[0].find('a',class_ = 'posLink',href=True)['href']

'https://th.jobsdb.com/th/en/job/internal-control-manager-300003002203716'

In [24]:
# Get all job links for Accounting --> Audit page 1
job_links = soup.find_all('div',class_="result-sherlock-cell branded")
job_links_plus = soup.find_all('div',class_="result-sherlock-cell branded plus")
all_job_links = job_links_plus + job_links

In [25]:
#Show some job links for Accounting --> Audit page 1
for job_link in all_job_links[0:5]:
    print(job_link.find('a',class_='posLink',href=True)['href'])

https://th.jobsdb.com/th/en/job/internal-control-manager-300003002203716
https://th.jobsdb.com/th/en/job/audit-manager-300003002203605
https://th.jobsdb.com/th/en/job/finance-manager-300003002203583
https://th.jobsdb.com/th/en/job/senior-ar-officer-300003002203369
https://th.jobsdb.com/th/en/job/director-of-corporate-accounting-300003002203344


In [36]:
##Final Scraping
## จิงๆแล้วทำแบบเข้า job category ใหญ่ทีเดียวก้ได้ เช่นแบบ Accounting ทีเดียวพอไม่ต้อง accounting --> audit 
## เพราะว่ามันจะมีพวกลิ้งซ้ำแต่ไว้ทำทีหลังก้ได้มั้ง


jobs_df = pd.DataFrame(columns = ['job_title','job_categories','career_level',
                                  'experience_year','salary','employment_type','job_responsibilities',
                                 'job_qualifications','job_description'])

num_request_without_break = 0  # to prevent too frequent requests
sheet_number = 1  # use to name excel sheet

for url in url_list:  # Iterate over all job categories
    
    ##Get first job page and the last page number
    url = re.sub('[0-9]','',url)
    source = requests.get(url).text
    soup = BeautifulSoup(source,'html.parser')
    last_page_url = soup.find_all('a',class_ = 'pagebox')
    last_page_url = last_page_url[-1]['href'] if last_page_url != [] else None
    max_page = int(re.findall(r'\b\d+\b', last_page_url)[0]) if last_page_url is not None else 0  # last page number

    for page in range(max_page):  # iterate over all page until all page has been reached
        page = page+1
        if page == 1:
            pass
        else:
            url = re.sub('[0-9]','',url)
            url = url + str(page)
            source = requests.get(url).text
            soup = BeautifulSoup(source,'html.parser')  
        job_links = soup.find_all('div',class_="result-sherlock-cell branded")
        job_links_plus = soup.find_all('div',class_="result-sherlock-cell branded plus")
        all_job_links = job_links_plus + job_links
        print(url)    
        
        #request counter (request for a page)
        num_request_without_break += 1
    
        for i,job_link in enumerate(all_job_links):  ## Get all the jobs for each page
            
            # open the job link page
            url_job = job_link.find('a',class_='posLink',href=True)['href']
            source = requests.get(url_job).text
            soup = BeautifulSoup(source,'html.parser')

            # get career level and experience year
            career_level = soup.find('b',class_ = 'primary-meta-lv')
            experience_year = soup.find('b',class_ = 'primary-meta-exp')
            
            career_level = career_level.text if career_level is not None else None
            experience_year = experience_year.text if experience_year is not None else None
            
            # get all job categories 
            job_function_html = soup.find('div',class_="primary-meta-box row meta-jobfunction")
            job_function_html = job_function_html.find('p').find_all('a') if job_function_html is not None else []
            job_categories = []
            for html in job_function_html:
                if 'job-list' in html['href']:
                    job_categories.append(re.compile("job-list/(.*?)/(.*?)/").findall(html.prettify())[0])
            
            
            #Get job responsibilities & qualifications & job description
            job_description = soup.find_all('div',class_="jobad-primary-details")
            job_description = job_description[0].text if job_description != [] else ''
            job_description = job_description.replace("\n","")
            job_description = job_description.replace("\xa0","")
            job_description = job_description.replace("\r","")
            job_responsibilities = re.compile("(.*)Qualification").findall(job_description) #All before Qualification
            job_qualifications = re.compile("Qualification(.*)").findall(job_description) # All after Qualification

                    
            # get job_title,salary,employment type   

            
            job_title = soup.find('h1',class_="general-pos ad-y-auto-txt2")
            job_title = job_title.text if job_title is not None else None
            
            salary = soup.find('div',class_="primary-meta-box row meta-salary")
            salary = salary.find('span').text if salary is not None else None
            
            employment_type = soup.find('div',class_="primary-meta-box row meta-employmenttype")
            employment_type = employment_type.find('p').text if employment_type is not None else None
            
            
            
            jobs_df.loc[len(jobs_df)] = [job_title,job_categories,career_level,
                                         experience_year,salary,employment_type,
                                        job_responsibilities,job_qualifications,
                                        job_description]
            
            #request counter (request for a job)
            num_request_without_break += 1
            
            # sleep and save checkpoint after ~1000 requests 
            if (num_request_without_break > 1000):    
                sheet_name = 'jobs_data'+str(sheet_number)+'.csv'
                jobs_df.to_csv(sheet_name,index = False,encoding = 'utf-8-sig')
                num_request_without_break = 0
                sheet_number += 1
                print('Excel sheet ',sheet_name,' saved')
                print('Sleeping for 30 secs')
                jobs_df = pd.DataFrame(columns = ['job_title','job_categories','career_level',
                                                  'experience_year','salary','employment_type','job_responsibilities',
                                                  'job_qualifications','job_description'])
                time.sleep(30)
            


jobs_df    

https://th.jobsdb.com/th/job-list/accounting/audit/
https://th.jobsdb.com/th/job-list/accounting/audit/2
https://th.jobsdb.com/th/job-list/accounting/audit/3
https://th.jobsdb.com/th/job-list/accounting/audit/4
https://th.jobsdb.com/th/job-list/accounting/audit/5
https://th.jobsdb.com/th/job-list/accounting/credit-control/
https://th.jobsdb.com/th/job-list/accounting/credit-control/2
https://th.jobsdb.com/th/job-list/accounting/credit-control/3
https://th.jobsdb.com/th/job-list/accounting/credit-control/4
https://th.jobsdb.com/th/job-list/accounting/general-accounting/
https://th.jobsdb.com/th/job-list/accounting/general-accounting/2
https://th.jobsdb.com/th/job-list/accounting/general-accounting/3
https://th.jobsdb.com/th/job-list/accounting/general-accounting/4
https://th.jobsdb.com/th/job-list/accounting/general-accounting/5
https://th.jobsdb.com/th/job-list/accounting/general-accounting/6
https://th.jobsdb.com/th/job-list/accounting/general-accounting/7
https://th.jobsdb.com/th/job

From cffi callback <function _verify_callback at 0x0000021F92B58828>:
Traceback (most recent call last):
  File "C:\Users\North\Anaconda3\envs\tf2\lib\site-packages\OpenSSL\SSL.py", line 311, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='th.jobsdb.com', port=443): Max retries exceeded with url: /th/en/job/senior-accountant-ap-%E0%B8%9E%E0%B8%99%E0%B8%B1%E0%B8%81%E0%B8%87%E0%B8%B2%E0%B8%99%E0%B8%9A%E0%B8%B1%E0%B8%8D%E0%B8%8A%E0%B8%B5%E0%B9%80%E0%B8%88%E0%B9%89%E0%B8%B2%E0%B8%AB%E0%B8%99%E0%B8%B5%E0%B9%89%E0%B8%AD%E0%B8%B2%E0%B8%A7%E0%B8%B8%E0%B9%82%E0%B8%AA-300003002201197 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

# Updated version

## Get all URL with last page number 

In [64]:
job_categories_list = job_category_df.job_category.unique().tolist()
url_list = []
for job_category in job_categories_list:
    url = 'https://th.jobsdb.com/th/jobs/'+job_category
    url_list.append(url)

url_list_with_max_page = []

# Iterate over all job category to get their last page
for url in url_list:
    source = requests.get(url).text
    soup = BeautifulSoup(source,'html.parser')
    last_page_url = soup.find_all('a',class_ = 'pagebox')
    last_page_url = last_page_url[-1]['href'] if last_page_url != [] else None
    max_page = int(re.findall(r'\b\d+\b', last_page_url)[0]) if last_page_url is not None else 0  # last page number
    url_list_with_max_page.append((url,max_page))
url_list_with_max_page[0:2]

[('https://th.jobsdb.com/th/jobs/accounting', 22),
 ('https://th.jobsdb.com/th/jobs/admin-hr', 19)]

## Request data

In [77]:
def get_job_links_in_page(main_url,page):
    request_url = main_url + '/' + str(page)
    source = requests.get(request_url).text
    soup = BeautifulSoup(source,'html.parser')
    job_links = soup.find_all('div',class_="result-sherlock-cell branded")
    job_links_plus = soup.find_all('div',class_="result-sherlock-cell branded plus")
    all_job_links = job_links_plus + job_links
    print(request_url)
    return all_job_links

In [85]:
request_counter = 0
sheet_number = 23

for main_page in url_list_with_max_page[-2:]:
    last_page = main_page[1]
    main_url = main_page[0]
    for page in range(last_page):
        page = page+1
        all_job_links = get_job_links_in_page(main_url,page)
        
        for i,job_link in enumerate(all_job_links):
            job_url = job_link.find('a',class_='posLink',href=True)['href']
            source = requests.get(job_url).text
            soup = BeautifulSoup(source,'html.parser')

            # get career level and experience year
            career_level = soup.find('b',class_ = 'primary-meta-lv')
            experience_year = soup.find('b',class_ = 'primary-meta-exp')
            
            career_level = career_level.text if career_level is not None else None
            experience_year = experience_year.text if experience_year is not None else None

            # get all job categories 
            job_function_html = soup.find('div',class_="primary-meta-box row meta-jobfunction")
            job_function_html = job_function_html.find('p').find_all('a') if job_function_html is not None else []
            job_categories = []
            for html in job_function_html:
                if 'job-list' in html['href']:
                    job_categories.append(re.compile("job-list/(.*?)/(.*?)/").findall(html.prettify())[0])
            
            
            #Get job responsibilities & qualifications & job description
            job_description = soup.find_all('div',class_="jobad-primary-details")
            job_description = job_description[0].text if job_description != [] else ''
            job_description = job_description.replace("\n","")
            job_description = job_description.replace("\xa0","")
            job_description = job_description.replace("\r","")
            job_responsibilities = re.compile("(.*)Qualification").findall(job_description) #All before Qualification
            job_qualifications = re.compile("Qualification(.*)").findall(job_description) # All after Qualification

                    
            # get job_title,salary,employment type   
            job_title = soup.find('h1',class_="general-pos ad-y-auto-txt2")
            job_title = job_title.text if job_title is not None else None
            
            salary = soup.find('div',class_="primary-meta-box row meta-salary")
            salary = salary.find('span').text if salary is not None else None
            
            employment_type = soup.find('div',class_="primary-meta-box row meta-employmenttype")
            employment_type = employment_type.find('p').text if employment_type is not None else None
            
            
            # save data into dataframe
            jobs_df.loc[len(jobs_df)] = [job_title,job_categories,career_level,
                                         experience_year,salary,employment_type,
                                        job_responsibilities,job_qualifications,
                                        job_description]
            
            #request counter (request for a job)
            request_counter += 1
            # sleep and save checkpoint after ~500 requests 
            if (request_counter % 500 == 0):    
                sheet_name = 'jobs_data'+str(sheet_number)+'.csv'
                jobs_df.to_csv(sheet_name,index = False,encoding = 'utf-8-sig')
                sheet_number += 1
                jobs_df = pd.DataFrame(columns = ['job_title','job_categories','career_level',
                                                  'experience_year','salary','employment_type','job_responsibilities',
                                                  'job_qualifications','job_description'])  # reset dataframe
 
                print('Excel sheet ',sheet_name,' saved')
                print('Sleeping for 60 secs')


                time.sleep(60)

https://th.jobsdb.com/th/jobs/transportation-logistics/1
https://th.jobsdb.com/th/jobs/transportation-logistics/2
https://th.jobsdb.com/th/jobs/transportation-logistics/3
https://th.jobsdb.com/th/jobs/transportation-logistics/4
https://th.jobsdb.com/th/jobs/transportation-logistics/5
https://th.jobsdb.com/th/jobs/transportation-logistics/6
https://th.jobsdb.com/th/jobs/transportation-logistics/7
https://th.jobsdb.com/th/jobs/transportation-logistics/8
https://th.jobsdb.com/th/jobs/transportation-logistics/9
https://th.jobsdb.com/th/jobs/transportation-logistics/10
https://th.jobsdb.com/th/jobs/others/1
https://th.jobsdb.com/th/jobs/others/2
https://th.jobsdb.com/th/jobs/others/3
https://th.jobsdb.com/th/jobs/others/4
Excel sheet  jobs_data23.csv  saved
Sleeping for 60 secs
https://th.jobsdb.com/th/jobs/others/5
https://th.jobsdb.com/th/jobs/others/6
https://th.jobsdb.com/th/jobs/others/7
https://th.jobsdb.com/th/jobs/others/8
https://th.jobsdb.com/th/jobs/others/9
https://th.jobsdb.com

In [ ]:
#Note : ต้องทำ inner join 22 กับ 23 เพราะมีrequest error จิงๆไม่ต้องก้ได้ ่oin แล้วก้ remove duplicate 5555

In [82]:
url_list_with_max_page[-1:]

[('https://th.jobsdb.com/th/jobs/others', 11)]

# Import data

In [88]:
import pandas as pd
import glob

path = r'C:\Users\North\my_app_folder\Data Scraping' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df_all = pd.concat(li, axis=0, ignore_index=True)
df_all.salary.value_counts()

           Salary provided                                                              6931
           Salary negotiable                                                            3345
Salary provided                                                                          266
Salary negotiable                                                                        226
           Base salary + high commission                                                 144
                                                                                        ... 
THB45,000 - 55,000 /month                                                                  1
           THB16,000 - 35,000 /month (negotiable)                                          1
           THB10,000 - 16,000 /month (includes high commission)                            1
THB70,000 - 90,000 /month                                                                  1
           THB13,000 - 25,000 /month (negotiable)                     

In [89]:
len(df_all)

12504

In [90]:
df_all.head()

,job_title,job_categories,career_level,experience_year,salary,employment_type,job_responsibilities,job_qualifications,job_description
0,HR Payroll Administrative(Junior Leve...,"[('admin-hr', 'administration-operation-cleric...",Entry Level,1 year experience,Salary provided,"Full Time, Permanent",[],[],Key Responsibilities include:To carry out all ...
1,Customer Service & Administrative Off...,"[('admin-hr', 'administration-operation-cleric...",Entry Level,NaN,Salary provided,"Full Time, Permanent",['Job DescriptionGiving information to clients...,['s:Bachelor’s degree in relevant discipline.0...,Job DescriptionGiving information to clients (...
2,Social Auditor - HR / Compensation & ...,"[('admin-hr', 'compensation-benefits'), ('admi...",Middle level,5 years experience,Salary provided,"Full Time, Permanent",[],[],JOB DUTIES:The following job duties are genera...
3,Senior Administrative & Secretary Exe...,"[('accounting', 'others'), ('admin-hr', 'admin...",Middle level,5 years experience,Salary provided,"Full Time, Permanent",[' ResponsibilitiesTo assist Director in all a...,"['sBachelor’s degree or higher in Accounting, ...",ResponsibilitiesTo assist Director in all adm...
4,HSE Auditor - ประสบการณ์ 5 ปี/ผู้ตรวจ...,"[('engineering', 'health-safety-environmental'...",Entry Level,5 years experience,Salary provided,"Full Time, Permanent",[],[],SPECIFIC RESPONSIBILITIES:Acting as the audit ...


In [93]:
df_all = df_all.drop_duplicates(subset =['job_title','job_categories','career_level','experience_year'])


In [103]:
df_all = df_all.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [109]:
df_all[ (df_all['salary']!= 'Salary provided') & (df_all['salary']!= 'Salary negotiable')]

,job_title,job_categories,career_level,experience_year,salary,employment_type,job_responsibilities,job_qualifications,job_description
17,HR Management & Development Officer - MRT Petc...,"[('admin-hr', 'compensation-benefits'), ('admi...",Middle level,NaN,"THB25,000 - 35,000 /month",Permanent,"['Location: MRT Petchaburi, BangkokSalary: 25,...","[""s:Bachelor's Degree in related fields. Male ...","Location: MRT Petchaburi, BangkokSalary: 25,00..."
18,Student Services Coordinator /เจ้าหน้าที่ฝ่ายป...,"[('admin-hr', 'administration-operation-cleric...",Entry Level,3 years experience,"THB20,000 - 30,000 /month (negotiable)",Full Time,[],[],Westminster International / WIN EducationDo yo...
26,Sales Coordinator (BTS Bangchak),"[('admin-hr', 'administration-operation-cleric...",Entry Level,NaN,"THB16,000 - 20,000 /month",Permanent,"['Location: BTS BangchakSalary: 15,000 - 20,00...","[""s:Female. Thai nationality. Bachelor's Degre...","Location: BTS BangchakSalary: 15,000 - 20,000 ..."
33,HR Generalist (Assistant Manager) - Retail Fu...,"[('admin-hr', 'administration-operation-cleric...",Middle level,NaN,"THB45,000 - 70,000 /month",Permanent,"["" Responsibilities:Supporting the recruitment...","[""s:Male or female. Thai nationality only. Age...",Responsibilities:Supporting the recruitment an...
41,C&B Manager - 120K up (Rayong),"[('admin-hr', 'compensation-benefits'), ('admi...",Top level,7 years experience,"THB90,000 - 120,000 /month (negotiable)","Full Time, Permanent",[''],"['Thai nationality, University degree in Human...","QualificationThai nationality, University degr..."
...,...,...,...,...,...,...,...,...,...
12451,Cloud Solution Consultant (Sales Cloud),"[('information-technology', 'it-consulting'), ...",Entry Level,NaN,"THB25,000 - 45,000 /month (negotiable)","Full Time, Permanent",[],[],Responsibilities:Maintaining and Establishing ...
12452,Programmer (25K-40K) @Samutprakarn (SDG-28740),"[('information-technology', 'application-speci...",Entry Level,2 years experience,"THB25,000 - 45,000 /month","Full Time, Permanent",[],[],<Job Responsibilities>To Analyze and solve pro...
12461,NaN,"[('information-technology', 'application-speci...",Full Time,NaN,"THB45,000 - 55,000 /month",Full Time,"[' Location:Bangkok, Close to BTS/MRTSalary:Up...",['s:Graduated Bachelor’s Degree in Computer Sc...,"Location:Bangkok, Close to BTS/MRTSalary:Up to..."
12467,SAP (All Modules) Consultant to Senior Level/ ...,"[('information-technology', 'application-speci...",Senior level,5 years experience,"Above THB120,000 /month",Full Time,['Position:SAP (All Modules) Consultant to Sen...,"['s:Thai national, age between 30 - 43 years.M...",Position:SAP (All Modules) Consultant to Senio...


# Maybe try Selenium ?

In [1]:
jobs_df

NameError: name 'jobs_df' is not defined